In [20]:
import requests 
import datetime 

from langchain.agents import tool 
from langchain.pydantic_v1 import BaseModel, Field

class RetornTempArgs(BaseModel):
    latitude: float = Field(description="Latitude da localidade que buscamos a temperatura")
    longitude: float = Field(description="Longitute da localidade que buscamos a temperatura")
 
@tool(args_schema=RetornTempArgs)
def retorna_temperatura_atual(latitude: float, longitude: float):
    """Retorne a temperatura atual para uma dada coordenada"""
    
    url = 'https://api.open-meteo.com/v1/forecast'
    
    params = {
		"latitude": latitude,
		"longitude": longitude,
		"hourly": "temperatura_2m",
		"forecast_days": 1
	}
    
    res = requests.get(url, params=params)
    
    if res.status_code == 200:
        resultado = res.json()
        
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado["hourly"]["time"]]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))
        
        temp_atual = resultado["hourly"]["temperature_2m"][index_mais_prox]
        return temp_atual 
    return f"Request para Api {url} falhou: {res.status_code}"
    
	# raise Exception(f"Request para Api {url} falhou: {res.status_code}")

        

In [3]:
retorna_temperatura_atual

StructuredTool(name='retorna_temperatura_atual', description='Retorne a temperatura atual para uma dada coordenada', args_schema=<class '__main__.RetornTempArgs'>, func=<function retorna_temperatura_atual at 0x781a19dd3eb0>)

In [5]:
retorna_temperatura_atual.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude da localidade que buscamos a temperatura',
  'type': 'number'},
 'longitute': {'title': 'Longitute',
  'description': 'Longitute da localidade que buscamos a temperatura',
  'type': 'number'}}

In [21]:
retorna_temperatura_atual.invoke({"latitude": -30, "longitude": -50})

'Request para Api https://api.open-meteo.com/v1/forecast falhou: 400'

In [26]:
import wikipedia 
wikipedia.set_lang('pt')

@tool 
def busca_wikipedia(query: str):
    """Faz busca no wikipedia e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f"Título da página: {titulo}\nResumo: {wiki_page.summary}")
        except:
            pass
        
    if not resumos:
        return "Busca não teve retorno"
    
    return "\n\n".join(resumos)

In [27]:
busca_wikipedia.invoke({"query":"isaac asimov"})

'Título da página: Isaac Asimov\nResumo: Isaac Asimov (em russo: Исаак Юдович Озимов; romaniz.: Isaak Yudavich Azimov; Petrovichi, Rússia Soviética, atual Rússia, 2 de janeiro de 1920 — Brooklyn, 6 de abril de 1992) foi um escritor e bioquímico russo-americano, autor de obras de ficção científica e divulgação científica.\nAsimov é considerado um dos mestres da ficção científica e, junto com Robert A. Heinlein e Arthur C. Clarke, foi considerado um dos "três grandes" dessa área da literatura. A obra mais famosa de Asimov é a Série da Fundação, também conhecida como Trilogia da Fundação, que faz parte da série do Império Galáctico e que logo combinou com a Série Robôs. Também escreveu obras de mistério e fantasia, assim como uma grande quantidade de não-ficção. No total, escreveu ou editou mais de 500 volumes, aproximadamente 90 000 cartas ou postais, e tem obras em cada categoria importante do sistema de classificação bibliográfica de Dewey, exceto em filosofia.\nA maioria de seus livro

In [34]:
# from langchain.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI
# from langchain_core.utils.function_calling import convert_to_openai_function

# prompt = ChatPromptTemplate.from_messages([
#     ('system', 'Você é um assistente amigável chamado Isaac'),
#     ('user', '{input}')
# ])
# chat = ChatOpenAI()

# tools = [convert_to_openai_function(busca_wikipedia), convert_to_openai_function(retorna_temperatura_atual)]
# chain = prompt | chat.bind(functions=tools)

from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
	("system", "Você é um assistente amigável chamada Isaac"),
	("user", "{input}")
])

chat = ChatOpenAI()

tools = [convert_to_openai_function(busca_wikipedia), convert_to_openai_function(retorna_temperatura_atual)]
chain = prompt | chat.bind(functions=tools)

In [35]:
chain.invoke({"input": "Olá"})

AIMessage(content='Olá! Como posso ajudar você hoje?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 129, 'total_tokens': 141}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c4c72f7f-450e-4c89-9dc0-20638df7de24-0', usage_metadata={'input_tokens': 129, 'output_tokens': 12, 'total_tokens': 141})

In [36]:
chain.invoke({"input":"Qual é a temperatura de Porto Alegre?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-30.0346,"longitude":-51.2177}', 'name': 'retorna_temperatura_atual'}}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 137, 'total_tokens': 165}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-6ce800e3-d718-40cf-9a9e-df630cb476d3-0', usage_metadata={'input_tokens': 137, 'output_tokens': 28, 'total_tokens': 165})